## 0. Setup

In [ ]:
import torch
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
from EIANN import Network
import EIANN.utils as ut
import EIANN.plot as pt

pt.update_plot_defaults()


# Load dataset
tensor_flatten = T.Compose([T.ToTensor(), T.Lambda(torch.flatten)])
MNIST_train_dataset = torchvision.datasets.MNIST(root='../datasets/MNIST_data/', train=True, download=False,
                                         transform=tensor_flatten)
MNIST_test_dataset = torchvision.datasets.MNIST(root='../datasets/MNIST_data/',
                                        train=False, download=False,
                                        transform=tensor_flatten)

# Add index to train & test data
MNIST_train = []
for idx,(data,target) in enumerate(MNIST_train_dataset):
    target = torch.eye(len(MNIST_train_dataset.classes))[target]
    MNIST_train.append((idx, data, target))
    
MNIST_test = []
for idx,(data,target) in enumerate(MNIST_test_dataset):
    target = torch.eye(len(MNIST_test_dataset.classes))[target]
    MNIST_test.append((idx, data, target))
    
# Put data in dataloader
data_generator = torch.Generator()
train_sub_dataloader = torch.utils.data.DataLoader(MNIST_train[0:10000], shuffle=True, generator=data_generator)
val_dataloader = torch.utils.data.DataLoader(MNIST_train[-10000:], batch_size=10000, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(MNIST_test, batch_size=10000, shuffle=False)

epochs = 1
data_seed = 0
network_seed = 42

### Backprop (no weight contraints)

In [ ]:
# Create network
network_config = ut.read_from_yaml('../config/MNIST/EIANN_1_hidden_mnist_backprop_relu_SGD_config.yaml')
layer_config = network_config['layer_config']
projection_config = network_config['projection_config']
training_kwargs = network_config['training_kwargs']

bp_network = Network(layer_config, projection_config, seed=network_seed, **training_kwargs)

# Train network
data_generator.manual_seed(data_seed)

bp_network.train_and_validate(train_sub_dataloader, 
                              val_dataloader, 
                              epochs=epochs,
                              val_interval=(0, -1, 100),
                              store_history=False, 
                              store_weights=True,
                              store_weights_interval=(0, -1, 100),
                              status_bar=True)

In [ ]:
pt.plot_batch_accuracy(bp_network, test_dataloader, title='Final')
pt.plot_train_loss_history(bp_network)
pt.plot_validate_loss_history(bp_network)
pt.evaluate_test_loss_history(bp_network, test_dataloader, plot=True)

# pt.plot_MNIST_examples(bp_network, test_dataloader)
plt.show()

### Backprop (Dale's Law)

In [ ]:
# Create network
network_config = ut.read_from_yaml('../optimize/data/20221219_EIANN_1_hidden_mnist_backprop_Dale_relu_SGD_config.yaml')
layer_config = network_config['layer_config']
projection_config = network_config['projection_config']
training_kwargs = network_config['training_kwargs']

bpDale_network = Network(layer_config, projection_config, seed=network_seed, **training_kwargs)

# Train network
data_generator.manual_seed(data_seed)
bpDale_network.train_and_validate(train_sub_dataloader, 
                              val_dataloader, 
                              epochs=epochs,
                              val_interval=(0, -1, 100),
                              store_history=False, 
                              store_weights=True,
                              store_weights_interval=(0, -1, 100),
                              status_bar=True)

In [ ]:
pt.plot_batch_accuracy(bpDale_network, test_dataloader, title='Final')
pt.plot_train_loss_history(bpDale_network)
pt.plot_validate_loss_history(bpDale_network)
pt.evaluate_test_loss_history(bpDale_network, test_dataloader, plot=True)

# pt.plot_MNIST_examples(bpDale_network, test_dataloader)
plt.show()

In [ ]:
# Train network
bpDale_network.reset_history()

data_generator.manual_seed(data_seed)
bpDale_network.train(train_sub_dataloader, 
                     epochs=epochs, 
                     store_history=True, 
                     store_weights=False, 
                     status_bar=True)

In [ ]:
from EIANN.plot import *

fig,ax = plt.subplots(2,2,figsize=(12,4))

t = 0
hidden_activity = torch.mean(bpDale_network.H1.E.activity_history[t], dim=1)
output_activity = torch.mean(bpDale_network.Output.E.activity_history[t], dim=1)
ax[0,0].plot(hidden_activity,'--',color='r', label='H1E')
ax[0,0].plot(output_activity,color='r', label='OutputE')
ax[0,0].set_title('Before training')
ax[0,0].legend()

hiddenFBI_activity = torch.mean(bpDale_network.H1.FBI.activity_history[t], dim=1)
outputFBI_activity = torch.mean(bpDale_network.Output.FBI.activity_history[t], dim=1)
ax[1,0].plot(hiddenFBI_activity,'--',color='b', label='H1FBI')
ax[1,0].plot(outputFBI_activity,color='b', label='OutputFBI')


t = -1
hidden_activity = torch.mean(bpDale_network.H1.E.activity_history[t], dim=1)
output_activity = torch.mean(bpDale_network.Output.E.activity_history[t], dim=1)
ax[0,1].plot(hidden_activity,'--',color='r', label='H1E')
ax[0,1].plot(output_activity,color='r', label='OutputE')
ax[0,1].set_title('After training')
ax[0,1].legend()

hiddenFBI_activity = torch.mean(bpDale_network.H1.FBI.activity_history[t], dim=1)
outputFBI_activity = torch.mean(bpDale_network.Output.FBI.activity_history[t], dim=1)
ax[1,1].plot(hiddenFBI_activity,'--',color='b', label='H1FBI')
ax[1,1].plot(outputFBI_activity,color='b', label='OutputFBI')

plt.show()

## BTSP

In [ ]:
# Create network
network_config = ut.read_from_yaml('../optimize/data/20221219_EIANN_1_hidden_mnist_BTSP_config_D.yaml') 
layer_config = network_config['layer_config']
projection_config = network_config['projection_config']
training_kwargs = network_config['training_kwargs']

btsp_network = Network(layer_config, projection_config, seed=network_seed, **training_kwargs)

# Train network
data_generator.manual_seed(data_seed)

data_generator.manual_seed(data_seed)
btsp_network.train_and_validate(train_sub_dataloader, 
                              val_dataloader, 
                              epochs=epochs,
                              val_interval=(0, -1, 100),
                              store_history=False, 
                              store_weights=True,
                              store_weights_interval=(0, -1, 100),
                              status_bar=True)

In [ ]:
pt.plot_batch_accuracy(btsp_network, test_dataloader, title='Final')
pt.plot_train_loss_history(btsp_network)
pt.plot_validate_loss_history(btsp_network)
pt.evaluate_test_loss_history(btsp_network, test_dataloader, plot=True)

# pt.plot_MNIST_examples(btsp_network, test_dataloader)
plt.show()